# feature_extraction

In [3]:
# load library
import  sys,os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import gc
from collections import Counter
import copy

import warnings
warnings.filterwarnings('ignore')

# Countvector, TF-IDF feature_extraction library
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from scipy import sparse



 
%matplotlib inline

# 數據壓縮

In [ ]:
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2  #總共花多少MB
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes  # 各欄位資料類型
        if col_type in numerics:  # 欄位類型是數值型類型
            c_min = df[col].min() # 如果是數值型的話，找出最小的值
            c_max = df[col].max()  # 如果是數值型的話，找出最大的值
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)   # 如果欄位是屬於np.int8就將這欄位的資料類型轉換成int8
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else: # 欄位類型是字串型類型
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
                    
    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage before optimization is:{:.2f} MB".format(start_mem))
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
# load data
train_data = read_sql(train_sql)
test_data = read_sql(test_sql)
user_info = read_sql(user_info_sql).drop_duplicates()
user_log = read_sql(user_los_sql).rename(columns = {"seller_id" : 'merchant_id'})

train_data = reduce_mem_usage(train_data)
test_data = reduce_mem_usage(test_data)

user_info = reduce_mem_usage(user_info).drop_duplicates()
user_log = reduce_mem_usage(user_log)

# 合併資訊

## 處理Null和inf的資料

In [ ]:
# # 處理Null和inf的資料
def get_matrix(data):
    where_are_nan = np.isnan(data)
    where_are_inf = np.isnan(data)
    data[where_are_nan] = 0
    data[where_are_inf] = 0
    return data

In [ ]:
x_train = get_matrix(np.float_(x_train))

x_train = np.float_(x_train)

y_train = np.int_(y_train)


## 合併資訊

In [ ]:
list_join_func = lambda x: " ".join([str(i) for i in x])


agg_dict = {
            'item_id' : list_join_func,	
            'cat_id' : list_join_func,
            'seller_id' : list_join_func,
            'brand_id' : list_join_func,
            'time_stamp' : list_join_func,
            'action_type' : list_join_func
        }

rename_dict = {
            'item_id' : 'item_path',
            'cat_id' : 'cat_path',
            'seller_id' : 'seller_path',
            'brand_id' : 'brand_path',
            'time_stamp' : 'time_stamp_path',
            'action_type' : 'action_type_path'
        }


In [ ]:
user_log_path = user_log.groupby('user_id').agg(agg_dict).reset_index().rename(columns=rename_dict)
all_data = train_data.append(test_data)
all_data_path = train_data.merge(user_log_path,on='user_id')
gc.collect()
del user_log
gc.collect()

# 使用者歷程中，找出統計特徵extraction

## 統計特徵函數

In [ ]:
def cnt_(x):
    try:
        return len(x.split(' '))
    except:
        return -1

def nunique_(x):
    try:
        return len(set(x.split(' ')))
    except:
        return -1
    
def max_(x):
    try:
        return np.max([int(i) for i in x.split(' ')])
    except:
        return -1
    
def min_(x):
    try:
        return np.min([int(i) for i in x.split(' ')])
    except:
        return -1 
    
def std_(x):
    try:
        return np.std([float(i) for i in x.split(' ')])
    except:
        return -1 

def most_n(x, n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][0]
    except:
        return -1

def most_n_cnt(x, n):
    try:
        return Counter(x.split(' ')).most_common(n)[n-1][1]
    except:
        return -1   


def user_cnt(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(cnt_)
    return df_data

def user_nunique(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(nunique_)
    return df_data

def user_max(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(max_)
    return df_data


def user_min(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(min_)
    return df_data

def user_std(df_data, single_col, name):
    df_data[name] = df_data[single_col].apply(std_)
    return df_data

def user_most_n(df_data, single_col, name, n=1):
    func = lambda x: most_n(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data

def user_most_n_cnt(df_data, single_col, name, n=1):
    func = lambda x: most_n_cnt(x, n)
    df_data[name] = df_data[single_col].apply(func)
    return df_data



## extraction

In [ ]:
all_data = user_cnt(all_data,  'seller_path', 'user_cnt')
all_data = user_nunique(all_data,  'seller_path', 'seller_nunique')
all_data = user_nunique(all_data,  'cat_path', 'cat_nunique')
all_data = user_nunique(all_data,  'brand_path', 'brand_nunique')
all_data = user_nunique(all_data,  'item_path', 'item_nunique')
all_data = user_nunique(all_data,  'time_stamp_path', 'time_stamp_nunique')
all_data = user_nunique(all_data,  'action_type_path', 'action_type_nunique')

all_data = user_max(all_data,  'action_type_path', 'time_stamp_max')
all_data = user_min(all_data,  'action_type_path', 'time_stamp_min')
all_data = user_std(all_data,  'action_type_path', 'time_stamp_std')
all_data['time_stamp_range'] = all_data['time_stamp_max'] - all_data['time_stamp_min']

all_data = user_most_n(all_data, 'seller_path', 'seller_most_1', n=1)
all_data = user_most_n(all_data, 'cat_path', 'cat_most_1', n=1)
all_data = user_most_n(all_data, 'brand_path', 'brand_most_1', n=1)
all_data = user_most_n(all_data, 'action_type_path', 'action_type_1', n=1)
all_data = user_most_n_cnt(all_data, 'seller_path', 'seller_most_1_cnt', n=1)
all_data = user_most_n_cnt(all_data, 'cat_path', 'cat_most_1_cnt', n=1)
all_data = user_most_n_cnt(all_data, 'brand_path', 'brand_most_1_cnt', n=1)
all_data = user_most_n_cnt(all_data, 'action_type_path', 'action_type_1_cnt', n=1)


# 店家相關的特徵extraction


## 店家相關的特徵函數

In [ ]:
def col_cnt_(df_data, columns_list, action_type):
    try:
        data_dict = {}
        col_list = copy.deepcopy(columns_list)
        if action_type != None:
            col_list += ['action_type_path']

        for col in col_list:
            data_dict[col] = df_data[col].split(' ')
        path_len = len(data_dict[col])
        

        data_out = []
        for i_ in range(path_len):
            data_txt = ''
            for col_ in columns_list:
                if data_dict['action_type_path'][i_] == action_type:
                    data_txt += '_' + data_dict[col_][i_]
            data_out.append(data_txt)

        return len(data_out)  
    except:
        return -1

def col_nuique_(df_data, columns_list, action_type):
    try:
        data_dict = {}

        col_list = copy.deepcopy(columns_list)
        if action_type != None:
            col_list += ['action_type_path']

        for col in col_list:
            data_dict[col] = df_data[col].split(' ')
            

        path_len = len(data_dict[col])

        data_out = []
        for i_ in range(path_len):
            data_txt = ''
            for col_ in columns_list:
                if data_dict['action_type_path'][i_] == action_type:
                    data_txt += '_' + data_dict[col_][i_]
            data_out.append(data_txt)
        
        return len(set(data_out))
    except:
        return -1
    

def user_col_cnt(df_data, columns_list, action_type, name):
    df_data[name] = df_data.apply(lambda x: col_cnt_(x, columns_list, action_type), axis=1)
    return df_data

def user_col_nunique(df_data, columns_list, action_type, name):
    df_data[name] = df_data.apply(lambda x: col_nuique_(x, columns_list, action_type), axis=1)
    return df_data

## extraction

In [ ]:
all_data = user_col_cnt(all_data,  ['seller_path'], '0', 'user_cnt_0')

all_data = user_col_cnt(all_data,  ['seller_path'], '1', 'user_cnt_1')

all_data = user_col_cnt(all_data,  ['seller_path'], '2', 'user_cnt_2')

all_data = user_col_cnt(all_data,  ['seller_path'], '3', 'user_cnt_3')

all_data = user_col_nunique(all_data,  ['seller_path'], '0', 'seller_nunique_0')

all_data = user_col_cnt(all_data,  ['seller_path', 'item_path'], '0', 'user_cnt_0')

all_data = user_col_nunique(all_data,  ['seller_path', 'item_path'], '0', 'seller_nunique_0')




# 利用Countvector和TF-IDF提取特徵

In [ ]:
tfidfVec = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS, ngram_range=(1, 1), max_features=100)

columns_list = ['seller_path', 'cat_path', 'brand_path', 'action_type_path', 'item_path', 'time_stamp_path']

for i, col in enumerate(columns_list):
    all_data_test[col] = all_data_test[col].astype(str)
    tfidfVec.fit(all_data_test[col])
    data_ = tfidfVec.transform(all_data_test[col])
    if i == 0:
        data_cat = data_
    else:
        data_cat = sparse.hstack((data_cat, data_))
    print(data_cat.shape)

df_tfidf = pd.DataFrame(data_cat.toarray())

df_tfidf.columns = ['tfidf_' + str(i) for i in df_tfidf.columns]

all_data = pd.concat([all_data, df_tfidf],axis=1)
len(all_data.columns)


# 得到特徵萃取後的train和test資料

In [ ]:
# train data
x_train_data = all_data_test[~all_data_test['label'].isna()]


x_train = x_train_data.loc[:,features_columns].values

y_train = x_train_data.loc[:,'label'].values

# test data
x_test_data = all_data_test[all_data_test['label'].isna()]

x_test = x_test_data.loc[:,features_columns].values




